# Linear Algebra Refresher

## Lesson 2 - Vectors

In [1]:
## NOTE - using Decimal to eliminate errors with floating point arithmatic.
# BUT need to be careful -  function outputs as Decimal()

from math import sqrt, acos, pi
from decimal import Decimal, getcontext

getcontext().prec = 30

class Vector(object):
    
    CANNOT_NORMALIZE_ZERO_VECTOR_MSG = 'Cannot normalize the zero vector'
    
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(x) for x in coordinates])
            self.dimension = len(self.coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')

            
##############################
# Use list comprehensions to create functions for add, minus and *scalar
##############################
    def plus(self,v):
        new_coordinates = [x+y for x,y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)

    def minus (self,v):
        new_coordinates = [x-y for x,y in zip(self.coordinates, v.coordinates)]
        return Vector(new_coordinates)

    def times_scalar (self, c):
        c = Decimal(c)
        new_coordinates = [(c*x) for x in self.coordinates]
        return Vector(new_coordinates)

###############################
# Create functions for magnitude and normalisation 
###############################
    def magnitude(self):
        coordinates_squared = [x**2 for x in self.coordinates]
        return Decimal(sqrt(sum(coordinates_squared)))

    def normalized(self):
        try:
            magnitude = self.magnitude()
            return self.times_scalar((Decimal('1.0'))/magnitude)

        except ZeroDivisionError:
            raise Exception(self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG)
            
##############################
# Dot product and angle
##############################

    def dot(self, v):
        return Decimal(sum([x*y for x,y in zip(self.coordinates, v.coordinates)]))
    
    
    def angle_with(self, v, in_degrees=False):
        try:
            u1 = self.normalized()
            u2 = v.normalized()
            angle_in_radians = acos(round(float(u1.dot(u2)),4))
            
            if in_degrees:
                degrees_per_radian = 180./pi
                return angle_in_radians * degrees_per_radian
            else:
                return angle_in_radians
        
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception('Cannot compute an angle with theb zero vector')

#################################
# Parallel and Orthogonal
#################################

    def is_orthogonal_to(self, v, tolerance=1e-10):
        return abs(self.dot(v)) < tolerance
    
    def is_parallel_to(self,v):
        return ( self.is_zero() or
                 v.is_zero() or
                 self.angle_with(v) == 0 or
                 self.angle_with(v) == pi)
    
    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance

    
#################################
# Vector Projections
#################################
    def component_parallel_to(self, basis):
        try:
            u = basis.normalized()
            weight = self.dot(u)
            return u.times_scalar(weight)
        except Exception as e:
            if str(e) == self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else:
                raise e
    
    
    def component_orthogonal_to(self, basis):
        try:
            projection = self.component_parallel_to(basis)
            return self.minus(projection)
        
        except Exception as e:
            if str(e)==self.NO_UNIQUE_PARALLEL_COMPONENT_MSG:
                raise Exception(self.NO_UNIQUE_PARALLEL_COMPONENT_MSG)
            else:
                raise e

                
#################################
# Cross Products
#################################                
    def cross(self,v):
        try:
            x_1, y_1, z_1 = self.coordinates
            x_2, y_2, z_2 = v.coordinates
            new_coordinates = [y_1*z_2 - y_2*z_1,
                              -(x_1*z_2 - x_2*z_1),
                               x_1*y_2 - x_2*y_1]
            return Vector(new_coordinates)
        
        except ValueError as e:
                msg = str(e)
                if msg == 'need more than 2 values to unpack':
                    self_embedded_in_R3 = Vector(self.coordinates + ('0',))
                    v_embedded_in_R3 = Vector(v.coordinates + ('0',))
                    return self_embedded_in_R3.cross(v_embedded_in_R3)
                elif (msg == 'too many values to unpack' or
                      msg == 'need more than 1 value to unpack'):
                    raise Exception(self.ONLY_DEFINED_IN_TWO_THREE_DIMS_MSG)
                else:
                    raise e
    
    
    def area_of_triangle_with(self, v):
        return self.area_of_parallelogram_with(v) / Decimal('2.0')
    
    
    def area_of_parallelogram_with(self, v):
        cross_product = self.cross(v)
        return cross_product.magnitude()
                
                
                
#################################
# from initial class definition
##################################
    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    
    def __iter__(self):
            return iter(self.coordinates)
        

    def __getitem__(self,index):
            return self.coordinates[index]

In [2]:
my_vector = Vector([1,2,3])
my_vector_2 = Vector([1,2,3])
my_vector_3 = Vector([-1,2,3])

my_vector_4 = Vector([1,2,3,4,5,6,7])

In [3]:
print(my_vector)
print(my_vector.coordinates)
print(my_vector.dimension)

print(my_vector_4.dimension)

Vector: (Decimal('1'), Decimal('2'), Decimal('3'))
(Decimal('1'), Decimal('2'), Decimal('3'))
3
7


In [4]:
print(my_vector == my_vector_2)

print(my_vector == my_vector_3)

True
False


## Plus, Minus, Scalar multiply

###### Use list comprehensions to create functions for add, minus and *scalar

```
def plus(self,v):
    new_coordinates = [x+y for x,y in zip(self.coordinates, v.coordinates)]
    return Vector(new_coordinates)

def minus (self,v):
    new_coordinates = [x-y for x,y in zip(self.coordinates, v.coordinates)]
    return Vector(new_coordinates)

def times_scalar (self, c):
    new_coordinates = [c*x for x in self.coordinates]
    return Vector(new_coordinates)
```  
    


In [5]:
vector_1 = Vector([8.218, -9.341])
vector_2 = Vector([-1.129, 2.111])

print(vector_1.plus(vector_2))

Vector: (Decimal('7.08899999999999996802557689080'), Decimal('-7.22999999999999909405801190587'))


In [6]:
vector3 = Vector([7.119, 8.215])
vector4 = Vector([-8.223, 0.878])

print(vector3.minus(vector4))

Vector: (Decimal('15.3420000000000005258016244625'), Decimal('7.33699999999999985522691758888'))


In [7]:
vector5=Vector(['1.671', '-1.012', '-0.318'])
scalar = 7.41

print(vector5.times_scalar(scalar))

Vector: (Decimal('12.3821100000000002374633822910'), Decimal('-7.49892000000000014381384971784'), Decimal('-2.35638000000000004519051799434'))


In [8]:
v3 = Vector([5.581, -2.136])

print  v3

Vector: (Decimal('5.58100000000000040500935938325710594654083251953125'), Decimal('-2.13600000000000012079226507921703159809112548828125'))


## Magnitude and direction

###### magnitude and normalised unit vector

```
from math import sqrt

def magnitude(self):
    coordinates_squared = [x**2 for x in self.coordinates]
    return sqrt(sum(coordinates_squared))

def normalized(self):
    try:
        magnitude = self.magnitude()
        return self.times_scalar(1./magnitude)
    
    except ZeroDivisionError:
        raise('Cannot normalise the zero vector')
    ```

In [9]:
vector1 = Vector([-0.221, 7.437])
vector2 = Vector([8.813, -1.331, -6.247])

print(vector1.magnitude())
print(vector2.magnitude())



7.4402829247280646285389593685977160930633544921875
10.8841875672922885343041343730874359607696533203125


In [10]:
vector3 = Vector([5.581, -2.136])
vector4 = Vector([1.996, 3.108, -4.554])

print(vector3.normalized())
print(vector4.normalized())

Vector: (Decimal('0.933935214086640295130539147343'), Decimal('-0.357442325262329983594964055642'))
Vector: (Decimal('0.340401295943301353537171045562'), Decimal('0.530043701298487295255023200306'), Decimal('-0.776647044952802835008995686630'))


## Inner Products (Dot Products)

In [11]:
v = Vector(['7.887', '4.138'])
w = Vector (['-8.802', '6.776'])
print (v.dot(w))

-41.382286


In [12]:
v = Vector(['-5.955', '-4.904', '-1.874'])
w = Vector (['-4.496', '-8.755', '7.103'])
print (v.dot(w))


56.397178


In [13]:
v = Vector(['3.183', '-7.627'])
w = Vector(['-2.668', '5.319'])
print(v.angle_with(w))

3.07229675739


In [14]:
v = Vector(['7.35', '0.221', '5.188'])
w = Vector(['2.751', '8.259', '3.985'])
print(v.angle_with(w, in_degrees=True))

60.2774829135


In [15]:
v = Vector([7.35, 0.221, 5.188])
w = Vector([2.751, 8.259, 3.985])
print(v.angle_with(w, in_degrees=True))

60.2774829135


## Parallel and Orthogonal Vectors

In [16]:
v = Vector([-7.579, -7.88])
w = Vector([22.737, 23.64])

print(v.angle_with(w, in_degrees = True))
print('Parallel?',v.is_parallel_to(w))
print('Orthogonal?', v.is_orthogonal_to(w))

180.0
('Parallel?', True)
('Orthogonal?', False)


In [17]:
v = Vector([-2.029, 9.97, 4.172])
w = Vector([-9.231, -6.639, -7.245])

print(v.angle_with(w, in_degrees = True))
print('Parallel?',v.is_parallel_to(w))
print('Orthogonal?', v.is_orthogonal_to(w))

121.600948112
('Parallel?', False)
('Orthogonal?', False)


In [18]:

v = Vector([-2.328, -7.284, -1.214])
w = Vector([-1.821, 1.072, -2.94])

print(round(v.dot(w),4))
print(v.angle_with(w, in_degrees = True))
print('Parallel?',v.is_parallel_to(w))
print('Orthogonal?', v.is_orthogonal_to(w))

-0.0
90.0
('Parallel?', False)
('Orthogonal?', True)


In [19]:
v = Vector([2.118, 4.827])
w = Vector([0,0])

#print(v.angle_with(w, in_degrees = True))
print('Parallel?',v.is_parallel_to(w))
print('Orthogonal?', v.is_orthogonal_to(w))

('Parallel?', True)
('Orthogonal?', True)


In [20]:
import numpy as np

v = np.array([-2.328, -7.284, -1.214],)
w = np.array([-1.821, 1.072, -2.94],)

#NOTE numpy.dot is dot product.
# np.arccos(dot.product) is angle in radians
# np.degrees converts radians to degrees
print(round(np.dot(v,w),4))
print(np.degrees(np.arccos(np.dot(v,w))))


-0.0
90.0


In [21]:
v = np.array([1,1,1])
w = np.array([1,1,0])

print(np.dot(v,w))
print(np.linalg.norm(v))
print(v/np.linalg.norm(v))

c = np.dot(v,w)/(np.linalg.norm(v) * np.linalg.norm(w))
angle = np.arccos(c)
print('Angle in radians',angle)
angle_deg = np.rad2deg(angle)
print('Angle in degress',angle_deg)

2
1.73205080757
[ 0.57735027  0.57735027  0.57735027]
('Angle in radians', 0.61547970867038748)
('Angle in degress', 35.264389682754661)


In [22]:
v = Vector([2,3,4])
w = Vector([4,5,6])

v.dot(w)

Decimal('47')

## Projecting Vectors

In [23]:
v = Vector([3.039, 1.879])
b = Vector([0.825, 2.036])

print(v.component_parallel_to(b))

Vector: (Decimal('1.08260696248446669921320880516'), Decimal('2.67174275832530224589459303452'))


In [24]:
v = Vector([-9.88, -3.264, -8.159])
b = Vector([-2.155, -9.353, -9.473])

print(v.component_orthogonal_to(b))

Vector: (Decimal('-8.35008104319576298139037182171'), Decimal('3.37606125428772042918826614535'), Decimal('-1.43374604278118531453872963265'))


In [25]:
v = Vector([3.009, -6.172, 3.692, -2.51])
b = Vector([6.404, -9.144, 2.759, 8.718])

print('parallel component',v.component_parallel_to(b))
print('orthogonal component',v.component_orthogonal_to(b))

('parallel component', <__main__.Vector object at 0x1032c4b10>)
('orthogonal component', <__main__.Vector object at 0x1032c4b10>)


In [26]:
## Implementing with numpy

v = np.array([3.039, 1.879])
b = np.array([0.825, 2.036])

vv = Vector([3.039, 1.879])
bv = Vector([0.825, 2.036])

print('normalized bv',bv.normalized())
print('vv dot normalized bv', vv.dot(bv.normalized()))
print('vv parallel projection to bv', vv.component_parallel_to(bv))
print('vv orthogonal projection to bv', vv.component_orthogonal_to(bv))

print('\n\nNOW IN NUMPY')
b_unit = b/np.linalg.norm(b)
print('normalised unit basis vector',b_unit)

print ('v dot normalized b', np.dot(v, b_unit))

projv = b_unit*np.dot(v, b_unit)
print('projection of v parallel to b',projv )

print ('projection orthogonal to b', v-projv)

('normalized bv', <__main__.Vector object at 0x1089da190>)
('vv dot normalized bv', Decimal('2.88274993745269854721460324839'))
('vv parallel projection to bv', <__main__.Vector object at 0x1089da1d0>)
('vv orthogonal projection to bv', <__main__.Vector object at 0x1089da250>)


NOW IN NUMPY
('normalised unit basis vector', array([ 0.37554661,  0.92680351]))
('v dot normalized b', 2.8827499374526986)
('projection of v parallel to b', array([ 1.08260696,  2.67174276]))
('projection orthogonal to b', array([ 1.95639304, -0.79274276]))


## Cross Product


In [27]:

v = Vector([8.462, 7.893, -8.187])
w = Vector([6.984, -5.975, 4.778])

print('v x w = ' , v.cross(w))
print(w.magnitude())

print('area of parallelogram =', v.area_of_parallelogram_with(w))


('v x w = ', <__main__.Vector object at 0x1032c4c50>)
10.3588689054355729268763752770610153675079345703125
('area of parallelogram =', Decimal('144.300032696633223849858040921390056610107421875'))


In [28]:

v = Vector([-8.987, -9.838, 5.031])
w = Vector([-4.268, -1.861, -8.866])

print('area of parallelogram =', v.area_of_parallelogram_with(w))


('area of parallelogram =', Decimal('142.1222214018463319007423706352710723876953125'))


In [29]:

v = Vector([1.5,9.547,3.691])
w = Vector([-6.007,0.124,5.772])

print('area of triangle =', v.area_of_triangle_with(w))

print('vector cross=', Vector.cross(v,w))

print('area of triangle =', Vector.area_of_triangle_with(v,w)) 


('area of triangle =', Decimal('42.5649373994189375025598565117'))
('vector cross=', <__main__.Vector object at 0x1032c4750>)
('area of triangle =', Decimal('42.5649373994189375025598565117'))


In [30]:
# In numpy ...

v = np.array([8.462, 7.893, -8.187])
w = np.array([6.984, -5.975, 4.778])

print('cross product=',np.cross(v,w))

print('area of parallelogram = ', np.linalg.norm(np.cross(v,w)))


('cross product=', array([ -11.204571,  -97.609444, -105.685162]))
('area of parallelogram = ', 144.30003269663322)
